In [3]:
# importing OpenCV, time and Pandas library
import cv2, time, pandas
from datetime import datetime


# Assigning our static_back to None
static_back = None
motion_list = [ None, None ]
time = []
df = pandas.DataFrame(columns = ["Start", "End"])
video = cv2.VideoCapture(0)

# Infinite while loop to treat stack of image as video
while True:
    check, frame = video.read()    
    motion = 0    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    
    
    if static_back is None:
        static_back = gray
        continue


    diff_frame = cv2.absdiff(static_back, gray)


    thresh_frame = cv2.threshold(diff_frame, 30, 255, cv2.THRESH_BINARY)[1]
    thresh_frame = cv2.dilate(thresh_frame, None, iterations = 2)

    cnts,_ = cv2.findContours(thresh_frame.copy(),
                    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in cnts:
        if cv2.contourArea(contour) < 10000:
            continue
        motion = 1

        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)


    motion_list.append(motion)

    motion_list = motion_list[-2:]

# Appending Start time of motion
    if motion_list[-1] == 1 and motion_list[-2] == 0:
        time.append(datetime.now())

# Appending End time of motion
    if motion_list[-1] == 0 and motion_list[-2] == 1:
        time.append(datetime.now())

# Displaying image in gray_scale
    cv2.imshow("Gray Frame", gray)
    cv2.imshow("Difference Frame", diff_frame)
    cv2.imshow("Threshold Frame", thresh_frame)
    cv2.imshow("Color Frame", frame)
    key = cv2.waitKey(1)
    
# if q entered whole process will stop
    if key == ord('q'):
        if motion == 1:
            time.append(datetime.now())
        break

# Appending time of motion in DataFrame
for i in range(0, len(time), 2):
    df = df.append({"Start":time[i], "End":time[i + 1]}, ignore_index = True)

# Creating a CSV file in which time of movements will be saved
df.to_csv("Time_of_movements.csv")
video.release()

# Destroying all the windows
cv2.destroyAllWindows()


In [4]:
import pandas as pd
df=pd.read_csv("Time_of_movements.csv")
df

,Unnamed: 0,Start,End
0,0,2021-06-15 18:37:08.656264,2021-06-15 18:37:10.449298
1,1,2021-06-15 18:37:13.064642,2021-06-15 18:37:14.084620
2,2,2021-06-15 18:37:19.523872,2021-06-15 18:37:21.152031
3,3,2021-06-15 18:37:25.948502,2021-06-15 18:37:27.118209
4,4,2021-06-15 18:37:30.350539,2021-06-15 18:37:31.501596
5,5,2021-06-15 18:37:33.774249,2021-06-15 18:37:34.815540
